# XGBoost on datasets that do not fit in memory
Let's explore some strategies for using ML methods such as XGBoost on datasets that does not fit in memory.

In [1]:
import h5py
import numpy as np
import pandas as pd

/home/sb1a.sparebank1.no/a501464/anaconda3/envs/xgb-big/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Let's create a h5py dummy dataset to play with. We'll use the (definitely not big data) numeric columns of the titanic dataset:

In [2]:
titanic_df = pd.read_csv("titanic_original.csv")
titanic_df = titanic_df[titanic_df["survived"] >= 0]

In [3]:
titanic_X_array = titanic_df.drop(["survived"], axis=1).select_dtypes(include="float64").as_matrix()
titanic_y_array = titanic_df["survived"].as_matrix()
print(titanic_X_array.shape, titanic_y_array.shape)
print(titanic_X_array.dtype, titanic_y_array.dtype)

(1309, 6) (1309,)
float64 float64


In [85]:
with h5py.File("titanic_h5py.hdf5", "w") as f:
    f.create_dataset("titanic_X", data=titanic_X_array)
    f.create_dataset("titanic_y", data=titanic_y_array)

Preparations complete! Now let's open the file we just made:

In [4]:
f = h5py.File('titanic_h5py.hdf5', 'r')

In [5]:
list(f)

['titanic_X', 'titanic_y']

In [6]:
X = f["titanic_X"]
y = f["titanic_y"]

In [7]:
X[0:2], y[0:2]

(array([[  1.    ,  29.    ,   0.    ,   0.    , 211.3375,      nan],
        [  1.    ,   0.9167,   1.    ,   2.    , 151.55  ,      nan]]),
 array([1., 1.]))

Cool. Now let's send our on-disk-stored X and y to XGBoost!

In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [11]:
model = XGBClassifier(silent=False, n_estimators=10)
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [12]:
model.predict(X[0:19,:]), y[0:19]

/home/sb1a.sparebank1.no/a501464/anaconda3/envs/xgb-big/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


(array([1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1.]),
 array([1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
        1., 1.]))

Haha ok that was easy! Let's start supersizing the dummy dataset to eventually larger-than-possible-to-fit-in-memory-size and see if it still works. First step: Let's see what happens when we go from ~1000 rows to ~1.000.000!

In [20]:
f.close()

In [21]:
with h5py.File("titanic_h5py_BIG.hdf5", 'w') as f:
    f.create_dataset("titanic_X", data=titanic_X_array, chunks=True, maxshape=(None,titanic_X_array.shape[1]))
    f.create_dataset("titanic_y", data=titanic_y_array, chunks=True, maxshape=(None,))
    
    i=0
    while i<1000:
        f["titanic_X"].resize(f["titanic_X"].shape[0] + titanic_X_array.shape[0], axis = 0)
        f["titanic_X"][-titanic_X_array.shape[0]:] = titanic_X_array
    
        f["titanic_y"].resize(f["titanic_y"].shape[0] + titanic_y_array.shape[0], axis = 0)
        f["titanic_y"][-titanic_y_array.shape[0]:] = titanic_y_array
        
        if i % 100 == 0:
            print(i)
        i += 1

0
100
200
300
400
500
600
700
800
900


Let's have a look:

In [22]:
f = h5py.File('titanic_h5py_BIG.hdf5', 'r')

In [23]:
list(f)

['titanic_X', 'titanic_y']

In [24]:
X = f["titanic_X"]
y = f["titanic_y"]

In [25]:
print(X.shape)
print(y.shape)

(1310309, 6)
(1310309,)


Let's see how XGBoost copes with this:

In [28]:
model_big = XGBClassifier(subsample=0.01, silent=False, n_estimators=10)

In [29]:
model_big.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.01)

Works fine, but this was still just a small file, 70ish MB. We must go bigger! Next step, let's try ~7 GB:

In [30]:
f.close()
with h5py.File("titanic_h5py_BIGGER.hdf5", 'w') as f:
    f.create_dataset("titanic_X", data=titanic_X_array, chunks=True, maxshape=(None,titanic_X_array.shape[1]))
    f.create_dataset("titanic_y", data=titanic_y_array, chunks=True, maxshape=(None,))
    
    i=0
    while i<100000:
        f["titanic_X"].resize(f["titanic_X"].shape[0] + titanic_X_array.shape[0], axis = 0)
        f["titanic_X"][-titanic_X_array.shape[0]:] = titanic_X_array
    
        f["titanic_y"].resize(f["titanic_y"].shape[0] + titanic_y_array.shape[0], axis = 0)
        f["titanic_y"][-titanic_y_array.shape[0]:] = titanic_y_array
        
        if i % 1000 == 0:
            print(i)
        i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [31]:
f = h5py.File('titanic_h5py_BIGGER.hdf5', 'r')
X = f["titanic_X"]
y = f["titanic_y"]
print(X.shape)
print(y.shape)

(130901309, 6)
(130901309,)


In [ ]:
model_bigger = XGBClassifier(subsample=0.0001, silent=False, n_estimators=10)
model_bigger.fit(X, y)

Doesn't seem to work! Bah. Next try: http://xgboost.readthedocs.io/en/latest/how_to/external_memory.html